# Asynchronous computations


#### Some resources

- https://medium.freecodecamp.org/a-guide-to-asynchronous-programming-in-python-with-asyncio-232e2afa44f6
- https://hackernoon.com/asynchronous-python-45df84b82434
- https://www.youtube.com/watch?v=iG6fr81xHKA
- https://www.youtube.com/watch?v=tSLDcRkgTsY
- https://www.youtube.com/watch?v=b6z2VB1JU3U

#### What is async programming:

It's a style of concurrent programming in which tasks release the CPU during waiting periods, so that other tasks can use it.

#### What you need for asyncronous computations (1) : Suspend and Resume

- Asynchronous functions need the ability to **suspend** and **resume**
- A function that is suspended can leave the CPU to another function. This is useful for example when a function enters a waiting period.

There are several ways to implement suspend/resume in Python:

- callback functions
- Generator functions
- Async/await (py 3.5+)

#### What you need for asyncronous computations (2): Schedulling asynchronous tasks

In order to benefit from functions beeing suspended and resumed (in a given thread) we need a way to call functions while other functions are suspended. 

We need adn scheduler that does the following:

- The scheduler is usually called "event loop". 
- The loop keeps track of all the running tasks.
- When a function is suspended, returns control to the loop, which then finds another function to start or resume. This is usually called cooperative multi-tasking.

In [65]:
import platform
import asyncio  
import time  
from datetime import datetime

print(platform.python_version())

3.6.1


In [68]:

### Check
async def custom_sleep():  
    print('SLEEP', datetime.now())
    time.sleep(1)
    
async def factorial(name, number):  
    f = 1
    for i in range(2, number+1):
        print('Task {}: Compute factorial({})'.format(name, i))
        await custom_sleep()
        f *= i
    print('Task {}: factorial({}) is {}\n'.format(name, number, f))

start = time.time() 
loop = asyncio.new_event_loop()
tasks = [  
    asyncio.ensure_future(factorial("A", 3)),
    asyncio.ensure_future(factorial("B", 4)),
]
loop.run_until_complete(asyncio.wait(tasks))  
loop.close()
end = time.time()  
print("Total time: {}".format(end - start))

RuntimeError: Event loop is closed

In [29]:
import asyncio
import time

def hello_world():  
    print("Hi")
    time.sleep(2)
    print("Bye")

In [35]:
%%time
[hello_world() for i in range(4)]

Hi
Bye
Hi
Bye
Hi
Bye
Hi
Bye
CPU times: user 8 ms, sys: 0 ns, total: 8 ms
Wall time: 8.01 s


[None, None, None, None]

### Making the for loop async

In [45]:
import asyncio

@asyncio.coroutine
def hello_world_sleep():  
    print("Hi")
    yield from asyncio.sleep(2)
    print("Bye")   


In [56]:
%%time
loop = asyncio.new_event_loop()
[loop.run_until_complete( hello_world_sleep()) for i in range(4)]

Hi
Bye
Hi
Bye
Hi
Bye
Hi
Bye
CPU times: user 12 ms, sys: 4 ms, total: 16 ms
Wall time: 8.02 s


#### async def syntax

In [53]:
import asyncio

async def hello_world_sleep2():
    print("Hi")
    await asyncio.sleep(2)
    print("Bye")   

In [60]:
%%time
loop = asyncio.new_event_loop()
[loop.run_until_complete( hello_world_sleep()) for i in range(4)]

Hi
Bye
Hi
Bye
Hi
Bye
Hi
Bye
CPU times: user 4 ms, sys: 8 ms, total: 12 ms
Wall time: 8.01 s


## CPU heavy tasks

- Long CPU intensive tasks must routinely release the CPU to avoid starving other tasks.
- This can be done by "sleeping" periodically, such as once per loop iteration.
- To tell the loop to return control back as soon as posible sleep for 0 seconds.
- Example: `await asyncio.sleep(0)`

### Example 1

In [3]:
async def hello_world_async():   
    print("Hi")
    time.sleep(2)
    print("Bye")

In [4]:
hello_world_async()

<coroutine object hello_world_async at 0x7f7b1d0fb0a0>

In [26]:
loop = asyncio.get_event_loop()

In [6]:
loop.run_until_complete(hello_world_async)

TypeError: A Future, a coroutine or an awaitable is required

In [7]:
loop.close()


In [16]:

import asyncio
import aiohttp

urls = ['http://www.google.com', 'http://www.yandex.ru', 'http://www.python.org']

@asyncio.coroutine
def call_url(url):
    print('Starting {}'.format(url))
    response = yield from aiohttp.get(url)
    data = yield from response.text()
    print('{}: {} bytes: {}'.format(url, len(data), data))
    return data

futures = [call_url(url) for url in urls]

loop = asyncio.get_event_loop()
loop.run_until_complete(asyncio.wait(futures))

RuntimeError: Event loop is closed